In [1]:
# make a data generator that yeild the features and labels
# we first need to cut the audio into frames
# then get the labels to fit those frames


In [15]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
# cut audio into frames
import os

import numpy as np
import pandas as pd
import random


from pitch_tracker.utils.constants import (HOP_LENGTH, N_FFT, N_MELS,
                                           PICKING_FRAME_SIZE,
                                           PICKING_FRAME_STEP,
                                           SAMPLE_RATE,
                                           STEP_FRAME, STEP_TIME,
                                           N_CLASS, PRE_MIDI_START)
from pitch_tracker.utils.audio import load_audio_mono
from pitch_tracker.utils import files
from pitch_tracker.utils import dataset


In [17]:
audio_path = '../medleydb/medleydb/data/Audio/Phoenix_ScotchMorris/Phoenix_ScotchMorris_MIX.wav'
signal, sample_rate = load_audio_mono(
    audio_path, SAMPLE_RATE, keep_channel_dim=True)
mel_features = dataset.extract_melspectrogram_feature(
    signal, N_FFT, HOP_LENGTH, N_MELS, SAMPLE_RATE, backend='librosa')
print(mel_features.shape)

stft_feature = dataset.extract_stft_feature(signal, N_FFT, HOP_LENGTH)
print(stft_feature.shape)


(1, 88, 15258)
(1, 513, 15258)


In [18]:
csv_path = '../content/gen_label/2560/Melody2_midi/AClassicEducation_NightOwl.csv'
label_dir = '../content/gen_label/2560/Melody2_midi/'
label_dict = dataset.create_label_dict_from_dir(label_dir)
for k, v in label_dict.items():
    print(k, v.shape)


Phoenix_SeanCaughlinsTheScartaglen (1032, 3)
MusicDelta_FusionJazz (386, 3)
PurlingHiss_Lolita (1187, 3)
MusicDelta_Punk (170, 3)
AlexanderRoss_GoodbyeBolero (1690, 3)
TheSoSoGlos_Emergency (649, 3)
Meaxic_TakeAStep (1040, 3)
TheDistricts_Vermont (870, 3)
Meaxic_YouListen (1682, 3)
DreamersOfTheGhetto_HeavyLove (857, 3)
MusicDelta_Zeppelin (149, 3)
Phoenix_ScotchMorris (650, 3)
MusicDelta_ChineseYaoZu (190, 3)
MusicDelta_Shadows (48, 3)
Phoenix_BrokenPledgeChicagoReel (1246, 3)
MusicDelta_Gospel (384, 3)
BrandonWebster_YesSirICanFly (668, 3)
AmarLal_Rest (160, 3)
FacesOnFilm_WaitingForGa (778, 3)
PortStWillow_StayEven (605, 3)
MusicDelta_Britpop (193, 3)
AlexanderRoss_VelvetCurtain (2384, 3)
NightPanther_Fire (664, 3)
MusicDelta_ChineseChaoZhou (319, 3)
MusicDelta_ChineseJiangNan (229, 3)
SweetLights_YouLetMeDown (796, 3)
InvisibleFamiliars_DisturbingWildlife (808, 3)
JoelHelander_ExcessiveResistancetoChange (338, 3)
MusicDelta_LatinJazz (306, 3)
MusicDelta_Grunge (288, 3)
ClaraBerryAn

In [19]:
dataset_paths = dataset.create_dataset_path_dict(label_dir)
dataset_paths_df = pd.DataFrame(dataset_paths).transpose()
# dataset_paths_df.columns=['track_id', 'label_path', 'audio_path']
display(dataset_paths_df.head(5))
del dataset_paths_df


/Users/tien.d/workspace/GITHUB/mono_pitch_tracker/medleydb/medleydb/multitrack.py:371: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  metadata = yaml.load(f_in)


,0,1
Phoenix_SeanCaughlinsTheScartaglen,../content/gen_label/2560/Melody2_midi/Phoenix...,/Users/tien.d/workspace/GITHUB/mono_pitch_trac...
MusicDelta_FusionJazz,../content/gen_label/2560/Melody2_midi/MusicDe...,/Users/tien.d/workspace/GITHUB/mono_pitch_trac...
PurlingHiss_Lolita,../content/gen_label/2560/Melody2_midi/Purling...,/Users/tien.d/workspace/GITHUB/mono_pitch_trac...
MusicDelta_Punk,../content/gen_label/2560/Melody2_midi/MusicDe...,/Users/tien.d/workspace/GITHUB/mono_pitch_trac...
AlexanderRoss_GoodbyeBolero,../content/gen_label/2560/Melody2_midi/Alexand...,/Users/tien.d/workspace/GITHUB/mono_pitch_trac...


In [10]:
feature_label_gen = dataset.create_feature_label_generator(
    dataset_path_dict=dataset_paths,
    sample_rate=SAMPLE_RATE,
    n_fft=N_FFT,
    n_mels=N_MELS,
    n_class=N_CLASS,
    hop_length=HOP_LENGTH,
    picking_frame_step=PICKING_FRAME_STEP,
    picking_frame_size=PICKING_FRAME_SIZE,
    step_frame=STEP_FRAME,
    step_time=STEP_TIME,
    dist_threshold=0.1,
    empty_threshold=0.3,
)
output_dir = '../content/pickled_database/'
passed_songs = dataset.write_feature_label_to_disk_by_frame(
    feature_label_gen, output_dir, categorize_by_subdir = True)
failed_songs = [label for label in dataset_paths if label not in passed_songs]
display(failed_songs)
del failed_songs

Failed to load audio with `torchaudio`, fallback to `librosa` Failed to process a packet. (Invalid data found when processing input). 
Falied to load AimeeNorwich_Child - error 1650549857


['AimeeNorwich_Child']

In [176]:
pickled_paths = list(files.list_file_paths_in_dir(output_dir))
sample_pickled_path = random.choice(pickled_paths)
feature, label = files.load_pickle(sample_pickled_path)

label_onsets:np.ndarray = label['onset']
label_durations:np.ndarray = label['duration']
label_pitches:np.ndarray = label['pitch']

print(sample_pickled_path)
print(feature.shape)
print(label_onsets.shape)
print(label_durations.shape)
print(label_pitches.shape)


../content/pickled_database/TheDistricts_Vermont.pkl
(37, 1050, 88)
(37, 210)
(37, 210)
(37, 210, 88)


In [177]:


# Pitch label is one-hot encoded
# decode the pitch label:
# - apply flatnonzero to get the index of the label
# - add premidistart value to get the correct pitch
label_pitches = np.apply_along_axis(func1d=np.flatnonzero, axis=2, arr=label['pitch']).squeeze(-1)
label_pitches[label_pitches>0] += PRE_MIDI_START
print(label_pitches.shape)

(37, 210)


In [178]:
# Note: currently, each frame overlapped its next frame by 50%
def merge_frame(batch, picking_frame_step=PICKING_FRAME_STEP):
    # last_frame = batch[-1].copy()
    merged = batch[:, :picking_frame_step]
    merged = np.append(merged, batch[-1:, picking_frame_step:], axis=0)
    return merged



In [179]:
label_onsets = merge_frame(label_onsets)
label_durations = merge_frame(label_durations)
label_pitches = merge_frame(label_pitches)

In [180]:
label_onsets = label_onsets.flatten().astype(int)
label_durations = label_durations.flatten().astype(int)
label_pitches = label_pitches.flatten()

In [181]:
print(f'{"id":<10}{"time":<10}{"onset":<10}{"duration":<10}{"pitch":<10}')
for i, (onset, duration, pitch) in enumerate(zip(label_onsets, label_durations, label_pitches)):
    f_time = STEP_TIME * i
    s = f'{i+1:<10}{f_time:<10,.3f}{onset:<10}{duration:<10}{pitch:<10}'
    print(s)

id        time      onset     duration  pitch     
1         0.000     0         0         0         
2         0.058     0         0         0         
3         0.116     0         0         0         
4         0.174     0         0         0         
5         0.232     0         0         0         
6         0.290     0         0         0         
7         0.348     0         0         0         
8         0.406     0         0         0         
9         0.464     0         0         0         
10        0.522     0         0         0         
11        0.580     0         0         0         
12        0.639     0         0         0         
13        0.697     0         0         0         
14        0.755     0         0         0         
15        0.813     0         0         0         
16        0.871     0         0         0         
17        0.929     0         0         0         
18        0.987     0         0         0         
19        1.045     0         0

In [200]:
import pandas as pd
from pitch_tracker.utils import files

f_time = np.arange(start=0, stop=label_onsets.shape[0],dtype=np.float32) * STEP_TIME

csv_dir = '../content/gen_csv_label/'
csv_name = files.get_file_name(sample_pickled_path)
csv_path = os.path.join(csv_dir, csv_name + '.csv')

os.makedirs(csv_dir, exist_ok=True)

df = pd.DataFrame(
    np.array([f_time, label_onsets, label_durations, label_pitches], dtype=object),
    index=['time', 'onset', 'duration', 'pitch'],
).T
df.to_csv(csv_path, float_format='%.3f')

In [7]:
sample_path = '../content/pickled_database/AlexanderRoss_VelvetCurtain/AlexanderRoss_VelvetCurtain_0.pkl'
data = files.load_pickle(sample_path)
data

In [29]:
feature_label_gen = dataset.create_feature_label_generator(
    dataset_path_dict=dataset_paths,
    sample_rate=SAMPLE_RATE,
    n_fft=N_FFT,
    n_mels=N_MELS,
    n_class=N_CLASS,
    hop_length=HOP_LENGTH,
    picking_frame_step=PICKING_FRAME_STEP,
    picking_frame_size=PICKING_FRAME_SIZE,
    step_frame=STEP_FRAME,
    step_time=STEP_TIME,
    dist_threshold=0.1,
    empty_threshold=0.3,
)

for label_name, feature_label_pairs in feature_label_gen:
    n_durs_pos = 0
    n_pitches = 0
    for i, (feature, (onsets, durations, pitches)) in enumerate(feature_label_pairs):
        n_durs_pos += np.count_nonzero(durations)
        n_pitches += np.count_nonzero(pitches)
    print(f'{label_name}: {n_durs_pos}/{210*(i+1)} \t {n_pitches}/{210*(i+1)}')

Phoenix_SeanCaughlinsTheScartaglen: 6547/6720 	 6720/6720
MusicDelta_FusionJazz: 3181/3990 	 3990/3990
PurlingHiss_Lolita: 5979/8610 	 8610/8610
MusicDelta_Punk: 672/1050 	 1050/1050
AlexanderRoss_GoodbyeBolero: 10880/14070 	 14070/14070
TheSoSoGlos_Emergency: 3120/5670 	 5670/5670
Meaxic_TakeAStep: 7644/9660 	 9660/9660
TheDistricts_Vermont: 5836/7770 	 7770/7770
Meaxic_YouListen: 9884/13860 	 13860/13860
DreamersOfTheGhetto_HeavyLove: 5289/9660 	 9660/9660
MusicDelta_Zeppelin: 483/630 	 630/630
Phoenix_ScotchMorris: 5839/6090 	 6090/6090
MusicDelta_ChineseYaoZu: 1353/1890 	 1890/1890
MusicDelta_Shadows: 792/1260 	 1260/1260
Phoenix_BrokenPledgeChicagoReel: 6041/6300 	 6300/6300
MusicDelta_Gospel: 2115/2520 	 2520/2520
BrandonWebster_YesSirICanFly: 3012/3570 	 3570/3570
AmarLal_Rest: 3373/4830 	 4830/4830
FacesOnFilm_WaitingForGa: 5573/6930 	 6930/6930
PortStWillow_StayEven: 3682/7140 	 7140/7140
MusicDelta_Britpop: 981/1260 	 1260/1260
AlexanderRoss_VelvetCurtain: 11452/17640 	 17640

/Users/tien.d/opt/anaconda3/envs/thesis/lib/python3.9/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


HezekiahJones_BorrowedHeart: 5217/8400 	 8400/8400
BigTroubles_Phantom: 3378/4830 	 4830/4830
HopAlong_SisterCities: 7343/9660 	 9660/9660


KeyboardInterrupt: 